In [3]:
#錯誤:原先上下皆有障礙物，但過了第一個障礙物後，上面的障礙物卻消失了
import pygame #引入Pygame庫，使我們可以使用Pygame的功能來開發遊戲
import random
import os
#遊戲變數(通常會用全大寫表示這個數值之後不更動)
FPS = 60 #遊戲每秒的幀數（幀/秒）
WIDTH = 800 #視窗寬度(像素)
HEIGHT = 500 #視窗高度(像素)

#(Red,Green,Blue)調色盤，可填0-255，數字愈大顏色愈重
BlueGray = (119, 136, 153)
Black = (0, 0, 0)
Red = (255, 0, 0)
Yellow = (255, 255, 0)
White = (255,255,255)

#遊戲初始化 and 創建視窗
pygame.init() #初始化Pygame庫，必須在使用Pygame之前執行
screen = pygame.display.set_mode((WIDTH, HEIGHT)) #創建遊戲窗口，使用WIDTH和HEIGHT設定的尺寸
pygame.display.set_caption("殺時小遊戲") #設定遊戲視窗標題
#固定(控制)遊戲速度，調整遊戲循環的速度。創建一個 Clock 對象，以便控制遊戲的速度，使其達到每秒 60 幀的目標。這有助於確保遊戲在不同的計算機上以相同的速度運行，提供更一致的遊戲體驗
clock = pygame.time.Clock()


#載入圖片
player_img = pygame.image.load(os.path.join("img", "satyr-Sheet 2-1.png")).convert()
background_img = pygame.image.load(os.path.join("img", "back.png")).convert()
stone_img = pygame.image.load(os.path.join("img", "stone.png")).convert()

#載入音效
break_sound  = pygame.mixer.Sound(os.path.join("mp3", "over.mp3"))
pygame.mixer.music.load(os.path.join("mp3","back.mp3"))
pygame.mixer.music.play(-1)

#遊戲狀態，用於標識遊戲是否結束的變數，初始為False表示遊戲未結束。game_over = False 用於初始化遊戲狀態，表示遊戲一開始是未結束的。當特定的遊戲條件達到時，通常會將 game_over 設置為 True 來表示遊戲的結束
game_over = False
#用於存儲遊戲得分的變數，初始得分為0
score = 0

#創建一個類別(Player)
class Player(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self) #建平面
        self.image = pygame.transform.scale(player_img, (36,38))
        self.image.set_colorkey(White)
        pygame.display.set_icon(self.image)
        self.rect = self.image.get_rect() #將平面框起來做定位
        #定位
        self.rect.x = 50
        self.rect.y = 450
        self.is_jumping = False #追蹤玩家是否在跳躍中
        self.jump_height = 0 #跳躍高度

    #設定jump函式
    def jump(self):
        if not self.is_jumping:
            self.is_jumping = True
            self.jump_height = 100

    #設定update函式
    def update(self):
        #回傳布林值的函式(判斷是否有按鍵被按下，有則布林值為ture；無則為false)
        key_pressed = pygame.key.get_pressed()
        #檢測按鍵事件，以實現跳躍
        if key_pressed[pygame.K_SPACE]:
            self.jump()

        if self.is_jumping:
            if self.jump_height > 0 and self.rect.y > 0:  #條件判斷檢查高度
                self.rect.y -= 10
                self.jump_height -= 10
            else:
                self.is_jumping = False
                self.jump_height = 0
        else:
            if self.rect.y < 450:
                self.rect.y += 10

#創建一個類別(Obstacle)
class Obstacle(pygame.sprite.Sprite):
    def __init__(self, x, y, is_flipped):
        pygame.sprite.Sprite.__init__(self) #建平面
        obstacle_img = pygame.image.load(os.path.join("img", "stone.png")).convert()
        self.image = pygame.transform.scale(obstacle_img, (50, 300))  # 調整大小
        self.image.set_colorkey(White)
        self.rect = self.image.get_rect() #將平面框起來做定位
        self.rect.x = x
        if is_flipped:
            self.image = pygame.transform.flip(self.image, False, True)
            self.rect.y = y - 300  #上方的障礙物
        else:
            self.rect.y = y
        self.speedx = 4

    def update(self):
        self.rect.x -= self.speedx
        if self.rect.left < -60:
            self.rect.x = WIDTH + random.randrange(200, 400)
            self.rect.y = random.randrange(300, 450)  #隨機高度
            self.speedx = 4

#創建sprite群組
all_sprites = pygame.sprite.Group()
player = Player() #前面設定好的小物件player
all_sprites.add(player) #將小物件加到sprite群組裡

#創建一個用於存放障礙物的sprite群組，這將用於碰撞檢測
obstacles = pygame.sprite.Group()

#創建下方的障礙物
for i in range(10):
    #設定位置和是否在畫面下方(預設位置是上方，則下方是"False")
    obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(300, 450), False)
    all_sprites.add(obstacle) #將障礙物加到總的sprite群組
    obstacles.add(obstacle) #將障礙物加到障礙物的sprite群組裡

#創建上方的障礙物
for i in range(10):
    #設定位置和是否在畫面上方
    obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(0, 150), True)
    all_sprites.add(obstacle) #將障礙物加到總的sprite群組
    obstacles.add(obstacle) #將障礙物加到障礙物的sprite群組裡

#創建字體物件，用於顯示分數
font = pygame.font.Font(None, 36) #(預設字體, 字體大小)

#遊戲迴圈迴圈初始化
running = True
while running:
    clock.tick(FPS) #1秒內迴圈只能被執行60(FPS)次
    #取得輸入
    for event in pygame.event.get(): #回傳輸入者的動作(ex:按下空白建)
        if event.type == pygame.QUIT: #如果用戶點擊視窗的關閉按鈕
            running = False #結束遊戲迴圈
        elif event.type == pygame.KEYDOWN: #如果用戶按下鍵盤按鍵
            if event.key == pygame.K_SPACE: #如果按下的是空格鍵
                if game_over: #如果遊戲結束狀態 #重設遊戲狀態
                    game_over = False
                    score = 0
                    all_sprites.empty() #清空所有精靈
                    player = Player() #創建新的玩家精靈
                    all_sprites.add(player)
                    obstacles.empty() #清空障礙物精靈群組

                    #創建新的障礙物
                    for i in range(10):
                        obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(300, 450), False)
                        all_sprites.add(obstacle)
                        obstacles.add(obstacle)

                    for i in range(10):
                        obstacle = Obstacle(WIDTH + random.randrange(200, 400), random.randrange(0, 150), True)
                        all_sprites.add(obstacle)
                        obstacles.add(obstacle)

    if not game_over:
        all_sprites.update() #執行群組內所有物件的update函式 #更新所有精靈的位置和狀態
        hits = pygame.sprite.spritecollide(player, obstacles, False)
        if hits:
            game_over = True #如果碰撞發生，遊戲結束
            break_sound.play()
    #畫面顯示
    background_img = pygame.transform.scale(background_img, (800,500))
    screen.blit(background_img, (0,0))
    all_sprites.draw(screen) #將sprite群組中所有物件都畫在螢幕上

    if game_over:
        pygame.mixer.music.load(os.path.join("mp3","back.mp3"))
        #pygame.mixer.music.load(os.path.join("mp3","back.mp3"))
        pygame.mixer.music.play(-1)

        text = font.render("Game Over! Press Space to Restart", True, Black)
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text, text_rect) #顯示遊戲結束提示
    else:
        score += 1
        score_text = font.render(f"Score: {score}", True, Black)
        screen.blit(score_text, (10, 10)) #顯示分數

    pygame.display.update() #畫面更新

pygame.quit() #退出遊戲